In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import re
from sklearn.pipeline import Pipeline
import nltk
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics import f1_score
from nltk.corpus import stopwords
import pickle
import tweepy
from IPython.display import clear_output
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

In [2]:
rf_df = pd.read_csv('data/full/rf_preproc_train.csv')

In [3]:
rf_df = rf_df[['user_id', 'avg_fav', 'avg_hash', 'avg_len', 'avg_ret', 'default_profile',
       'fake_words_score', 'favourites_count', 'followers_count', 'freq',
       'friends_count', 'genuine_words_score', 'listed_count', 'max_fav',
       'max_hash', 'max_len', 'max_ret', 'media_perc', 'min_fav', 'min_hash',
       'min_len', 'min_ret', 'porn_words_score',
       'profile_use_background_image', 'prop_words_score', 'quote_perc',
       'ret_perc', 'spam_words_score', 'statuses_count',
       'tweet_intradistance', 'url', 'url_intradistance', 'url_perc',
       'description_len', 'name_len', 'screen_name_len', 'age','target']]

In [4]:
X_train, X_val, y_train, y_val = train_test_split(rf_df.drop(columns=['target']), rf_df['target'], test_size=0.2, random_state=42)

In [5]:
X_train

,user_id,avg_fav,avg_hash,avg_len,avg_ret,default_profile,fake_words_score,favourites_count,followers_count,freq,...,spam_words_score,statuses_count,tweet_intradistance,url,url_intradistance,url_perc,description_len,name_len,screen_name_len,age
13903,68203632,0.000000,0.000000,37.000000,1.000000,1.0,0.000000,0,58,1.000000,...,0.000000,1,6.000000,0,0.000000,0.000000,0,9,13,9
11792,2353072477,0.100000,0.050000,67.200000,0.040000,0.0,0.018815,0,33,1.020408,...,0.081164,134,11.529610,1,0.000000,0.000000,104,13,13,4
16978,1175053682,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0,7,0.000000,...,0.000000,18,0.000000,0,0.000000,0.000000,154,15,15,5
14808,3291174194,0.020000,1.000000,130.770000,0.020000,1.0,0.013978,0,2,3.030303,...,0.025646,1380,10.132680,0,9.800000,1.000000,0,10,15,3
14934,2286005470,0.020000,2.430000,120.440000,0.020000,1.0,0.004864,0,30,0.232558,...,0.003762,8829,10.103471,0,9.750877,0.990000,0,15,11,4
5929,519435458,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,50,11,0.000000,...,0.000000,7,0.000000,0,0.000000,0.000000,0,15,8,6
24465,120082878,0.166667,0.060606,32.242424,6.712121,1.0,0.006172,5,51,0.023077,...,0.005585,25,8.436847,0,3.070084,0.151515,0,11,10,8
6404,211056956,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,439,32,0.000000,...,0.000000,84,0.000000,0,0.000000,0.000000,0,12,11,8
8952,819642960648294400,0.020000,0.100000,130.800000,4432.190000,1.0,0.016213,43377,1495,7.692308,...,0.038094,42759,15.203770,0,3.246413,0.110000,147,20,12,1
13887,67609542,0.000000,0.000000,65.000000,1.000000,1.0,0.000000,0,143,1.000000,...,0.059865,1,11.000000,1,0.000000,0.000000,0,8,12,9


In [6]:
rf = RandomForestClassifier(n_estimators=30, criterion='gini', max_depth=24) #max_depth=24

In [7]:
cv = cross_validate(rf, X_train.drop(columns=['user_id']), y_train, cv=10, scoring='f1_macro')

In [36]:
cv['test_score'].mean()

0.9566852854982768

In [37]:
rf.fit(X_train.drop(columns=['user_id']), y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=24, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [38]:
rf_pred = rf.predict_proba(X_val.drop(columns=['user_id']))

In [39]:
rf.predict_proba(X_val.drop(columns=['user_id']).iloc[0:1])

array([[7.99240506e-01, 0.00000000e+00, 0.00000000e+00, 2.00337553e-01,
        4.21940928e-04]])

In [40]:
X_val.iloc[0:1]['user_id']

0    610392907
Name: user_id, dtype: int64

In [41]:
rf_prob = pd.DataFrame(rf_pred, columns=['rf_0', 'rf_1', 'rf_2', 'rf_3', 'rf_4'])

In [42]:
X_val.reset_index(inplace=True, drop=True)

In [43]:
rf_prob.reset_index(inplace=True, drop=True)

In [44]:
y_val.reset_index(inplace=True, drop=True)

In [45]:
rf_prob['user_id'] = X_val['user_id']

In [46]:
rf_prob['target'] = y_val

## Random Forest df è pronto

In [47]:
users = pd.read_csv('data/full/train.csv')

In [48]:
users = users[['user_id','target']]

In [49]:
tweets = pd.concat([pd.read_csv('data/propaganda/tweets.csv', sep='\t')[['user_id','full_text']], \
          pd.read_csv('data/porn/tweets.csv', sep='\t')[['user_id','full_text']], \
          pd.read_csv('data/spam/tweets.csv', sep='\t')[['user_id','full_text']], \
          pd.read_csv('data/fake_followers/tweets.csv', sep='\t')[['user_id','full_text']], \
          pd.read_csv('data/genuine/tweets.csv', sep='\t')[['user_id','full_text']]])

/home/matteo/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/matteo/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,8,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/matteo/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,4,8,11,18,19,20,23,26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/matteo/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,8,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  int

In [50]:
tweets = tweets.drop_duplicates()

In [51]:
tweets = pd.merge(tweets, users, on='user_id')

In [52]:
mask = np.in1d(tweets.user_id, X_train.user_id)

In [53]:
val_mask = np.in1d(tweets.user_id, X_val.user_id)

In [54]:
nb_train = tweets[mask]

In [55]:
nb_val = tweets[val_mask]

In [56]:
def remove_rt(x):
    if 'RT @' in x:
        try:
            return x[x.find(':')+2:]
        except:
            return x
    else:
        return x

In [57]:
stop_words = stopwords.words('english')

def remove_stop(x):
    return [word for word in x.split() if word not in stop_words]

In [58]:
nb_train['full_text'] = nb_train['full_text'].apply(lambda x: remove_rt(x))
nb_train['full_text'] = nb_train['full_text'].apply(lambda x: re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', x))
nb_train['full_text'] = nb_train['full_text'].apply(lambda x: re.sub(r'[^\w\s]','',x))
nb_train['full_text'] = nb_train['full_text'].apply(lambda x: x.lower())
nb_train['full_text'] = nb_train['full_text'].apply(lambda x: remove_stop(x))
nb_train['full_text'] = nb_train['full_text'].astype(str)
nb_train = nb_train[nb_train['full_text']!='[]']

/home/matteo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/matteo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/matteo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [59]:
nb_val['full_text'] = nb_val['full_text'].apply(lambda x: remove_rt(x))
nb_val['full_text'] = nb_val['full_text'].apply(lambda x: re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', x))
nb_val['full_text'] = nb_val['full_text'].apply(lambda x: re.sub(r'[^\w\s]','',x))
nb_val['full_text'] = nb_val['full_text'].apply(lambda x: x.lower())
nb_val['full_text'] = nb_val['full_text'].apply(lambda x: remove_stop(x))
nb_val['full_text'] = nb_val['full_text'].astype(str)
nb_val = nb_val[nb_val['full_text']!='[]']

/home/matteo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/matteo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/matteo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [60]:
stemmer = SnowballStemmer("english", ignore_stopwords=True)

class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(CountVectorizer, self).build_analyzer()
        return lambda doc:(stemmer.stem(w) for w in analyzer(doc))


stem_vectorizer = StemmedCountVectorizer(stemmer)

pipeline = Pipeline([('vect', stem_vectorizer), 
                     ('tfidf', TfidfTransformer()), 
                     ('mnb', MultinomialNB(fit_prior=False))])

clf = pipeline.fit(nb_train['full_text'], nb_train['target'])

In [61]:
predictions = []
i = 0

for usr in X_val['user_id']:
    
    i+=1
    clear_output()
    print(i)
    tweets_list = nb_val[nb_val.user_id == usr]['full_text']
    tweets_list = tweets_list[tweets_list.notnull()]
    if len(tweets_list[tweets_list.notnull()]) != 0:
        pred = clf.predict_proba(tweets_list)
        predictions.append(np.mean(pred, axis=0))
    else:
        predictions.append(np.array([0.2,0.2,0.2,0.2,0.2]))

1248


KeyboardInterrupt: 

In [ ]:
prob = pd.DataFrame(predictions, columns=['nb_0', 'nb_1', 'nb_2', 'nb_3', 'nb_4'])

In [ ]:
prob['target'] = y_val

In [ ]:
prob['user_id'] = X_val['user_id']

In [ ]:
# prob.to_csv('data/full/predictions1.csv')

In [ ]:
stack_df = rf_prob.merge(prob, on=['user_id' , 'target'])

In [ ]:
stack_df = stack_df.reindex_axis(sorted(stack_df.columns), axis=1)

In [ ]:
stack_df.head()

In [ ]:
# stack_df.to_csv('data/full/logreg_train.csv')

In [ ]:
stack_df

## Metamodel training

In [15]:
stack_df = pd.read_csv('data/full/logreg_train.csv').drop(columns=['Unnamed: 0'])

In [29]:
lr = LogisticRegression(penalty='l2', solver='saga', max_iter=1000, class_weight='balanced')

In [30]:
cv = cross_validate(lr, stack_df.drop(columns=['user_id', 'target']), stack_df['target'], cv=10, scoring='f1_macro')

In [32]:
cv['test_score'].mean()

0.9624199812349845

In [33]:
lr.fit(stack_df.drop(columns=['user_id', 'target']), stack_df['target'])

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=1000,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='saga', tol=0.0001, verbose=0, warm_start=False)

In [34]:
z = lr.predict_proba(stack_df.drop(columns=['user_id', 'target']).iloc[1:2])

In [35]:
pickle.dump( lr, open( "../scripts/lr.model", "wb" ) )

In [21]:
z = z.reshape(xx0.shape)
plt.figure(1, figsize=(12, 9))
plt.pcolormesh(xx0, xx1, z, cmap=background_cmap)

font = {'family' : 'sans', 'size'   : 32}
plt.rc('font', **font)

for i, color in zip(logistic.classes_, colors):
    idx = np.where(y == i)
    plt.scatter(x[idx, 0], x[idx, 1], c=color) #, cmap=plt.cm.Pastel2)

plt.xlabel('Sepal length')
plt.ylabel('Sepal width')

plt.xlim(xx0.min(), xx0.max())
plt.ylim(xx1.min(), xx1.max())
plt.xticks(())
plt.yticks(())


### plot also the planes
coef = logistic.coef_
intercept = logistic.intercept_

def plot_hyperplane(c, color):
    def line(x0):
        return (-(x0 * coef[c, 0]) - intercept[c]) / coef[c, 1]
    plt.plot([x0_min, x0_max], [line(x0_min), line(x0_max)],
             ls="--", lw=4, color=color)

# colors = "rgb"
for i, color in zip(logistic.classes_, colors):
    plot_hyperplane(i, color)

plt.show()

NameError: name 'xx0' is not defined